# Main Robot Script

Imports

In [1]:
# Line following
import lfrobot

# Camera
import traitlets
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

# Model
import numpy as np
import pickle
import time
from IPython.display import clear_output

Initialisation

In [2]:
import cv2

# def adjust_hue_bgr8_to_jpeg(frame, hue_shift=0):
#     # Convert BGR to HSV
#     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#     # Modify the hue channel (H channel)
#     hsv[..., 0] = (hsv[..., 0].astype(int) + hue_shift) % 10
#     # Convert back to BGR
#     bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
#     # Convert BGR to JPEG bytes
#     ret, jpeg = cv2.imencode('.jpg', bgr)
#     return jpeg.tobytes()

# Initialise camera
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)
print('Camera initialised')

# Load sign detection model
model = pickle.load('cnn_model.h5')
print('Sign detection model loaded')

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Camera initialised
Sign detection model loaded


Operation

In [ ]:
# Initialise line following
lfrobot.lfInit()
print('\nLine following initialised')

lfrobot.lfStart()
lfrobot.lfSpeed(0.1)
turnSpeed = 0.25 #turn speeds between 0.15 and 0.25 work best
lfrobot.lfTurnSpeed(turnSpeed)

current_speed = 0.1

# Using Variabls to store the state
slowState = False
stopState = False



while True:
    import cv2
    import numpy as np

    ## CAMERA INPUT
    test_img = camera.value

    # Resize to CNN input size
    test_img_resized = cv2.resize(test_img, (64, 64))

    # Convert grayscale to RGB if needed
    if test_img_resized.ndim == 2:
        test_img_resized = np.stack([test_img_resized]*3, axis=-1)

    # Normalize and add batch dimension
    test_array = np.expand_dims(test_img_resized, axis=0) / 255.0

    ## PREDICTION
    prediction = model.predict(test_array)[0]  # prediction vector
    prediction_index = (prediction > 0.75).nonzero()[0]  # high confidence indices

    ## ACTION
    if 1 in prediction_index:   # sheep
        print('sheep')
#         lfrobot.lfStop()
        lfrobot.lfSpeed(0)

    elif 0 in prediction_index: # green
        print('green')
        lfrobot.lfSpeed(0.1)
        pass

    elif 2 in prediction_index: # slow
        print('slow')
        slowState = True
        stopState = False

    elif 3 in prediction_index: # stop
        print('stop')
        stopState = True
#         lfrobot.lfStop()
        lfrobot.lfSpeed(0)
        time.sleep(5)
        lfrobot.lfSpeed(0.1)    
        time.sleep(1)

    elif 4 in prediction_index: # yellow
        print('yellow')
#         lfrobot.lfStop()
        lfrobot.lfSpeed(0)

    else:                       # free
        print('free')
        if slowState and not stopState:
            lfrobot.lfSpeed(0.05)
        else:
            lfrobot.lfSpeed(0.1)

    clear_output(wait=True)
    time.sleep(0.1)

speed0
0
stop
speed0.1
0.1


Deinitialise

In [ ]:
lfrobot.lfDeinit()

In [ ]:
camera_link.unlink()

In [ ]:
lfrobot.lfStop()